In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image


2024-03-31 13:07:23.404638: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pip install pandas numpy tensorflow Pillow


  Using cached pandas-2.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached pillow-10.2.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached pillow-10.2.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load metadata
train_metadata = pd.read_csv('/home/jovyan/shared/sampled_data/train_data.csv', usecols=['filename', 'age'])
val_metadata = pd.read_csv('/home/jovyan/shared/sampled_data/val_data.csv', usecols=['filename', 'age'])
test_metadata = pd.read_csv('/home/jovyan/shared/sampled_data/test_data.csv', usecols=['filename', 'age'])

# Data directories
train_image_dir = '/home/jovyan/shared/sampled_data/train_data'
val_image_dir = '/home/jovyan/shared/sampled_data/val_data'
test_image_dir = '/home/jovyan/shared/sampled_data/test_data'


In [5]:
# Image dimensions
image_size = (224, 224)
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_metadata,
    directory=train_image_dir,
    x_col='filename',
    y_col='age',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_metadata,
    directory=val_image_dir,
    x_col='filename',
    y_col='age',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_metadata,
    directory=test_image_dir,
    x_col='filename',
    y_col='age',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw')

Found 29913 validated image filenames.
Found 3762 validated image filenames.
Found 3739 validated image filenames.


In [6]:
# Increase data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Fine-tuning: Unfreeze some layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.3)(x)  # Dropout layer for regularization
predictions = Dense(1, activation='linear')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())

# Early stopping 
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

2024-03-31 13:07:33.022476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-31 13:07:33.087155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-31 13:07:33.088820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
# Train the model
model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping, lr_scheduler])

# Evaluate the model
loss = model.evaluate(test_generator)
print("Test Loss:", loss)


Epoch 1/50


2024-03-31 13:07:36.360979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-03-31 13:07:41.268920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2024-03-31 13:07:42.591949: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-31 13:07:43.060092: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55c4dea72090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-31 13:07:43.060150: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): GRID V100DX-16Q, Compute Capability 7.0
2024-03-31 13:07:43.070206: I tensorflow/compiler/mlir/tensorflow/utils/dum

935/935 [==============================] - ETA: 0s - loss: 847.2804

2024-03-31 13:09:03.311992: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


935/935 [==============================] - 113s 110ms/step - loss: 847.2804 - val_loss: 219.2500 - lr: 1.0000e-05
Epoch 2/50
935/935 [==============================] - 69s 73ms/step - loss: 189.0905 - val_loss: 161.5106 - lr: 1.0000e-05
Epoch 3/50
935/935 [==============================] - 69s 74ms/step - loss: 163.6791 - val_loss: 144.3600 - lr: 1.0000e-05
Epoch 4/50
935/935 [==============================] - 67s 72ms/step - loss: 146.6427 - val_loss: 130.8121 - lr: 1.0000e-05
Epoch 5/50
935/935 [==============================] - 69s 74ms/step - loss: 133.6914 - val_loss: 121.8690 - lr: 1.0000e-05
Epoch 6/50
935/935 [==============================] - 72s 77ms/step - loss: 122.9097 - val_loss: 115.5693 - lr: 1.0000e-05
Epoch 7/50
935/935 [==============================] - 73s 78ms/step - loss: 112.6119 - val_loss: 107.6998 - lr: 1.0000e-05
Epoch 8/50
935/935 [==============================] - 72s 77ms/step - loss: 105.5269 - val_loss: 102.2175 - lr: 1.0000e-05
Epoch 9/50
935/935 [=====

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



935/935 [==============================] - 70s 75ms/step - loss: 38.3311 - val_loss: 68.1009 - lr: 1.0000e-05
Epoch 30/50
935/935 [==============================] - 73s 78ms/step - loss: 36.2210 - val_loss: 65.9556 - lr: 1.0000e-05
Epoch 31/50
935/935 [==============================] - 72s 77ms/step - loss: 35.5239 - val_loss: 65.4489 - lr: 1.0000e-05
Epoch 32/50
935/935 [==============================] - 73s 79ms/step - loss: 34.3704 - val_loss: 67.4974 - lr: 1.0000e-05
Epoch 33/50
935/935 [==============================] - 71s 76ms/step - loss: 33.5571 - val_loss: 65.2825 - lr: 1.0000e-05
Epoch 34/50
935/935 [==============================] - 68s 72ms/step - loss: 33.4382 - val_loss: 68.2731 - lr: 1.0000e-05
Epoch 35/50
935/935 [==============================] - 70s 75ms/step - loss: 31.9122 - val_loss: 65.6197 - lr: 1.0000e-05
Epoch 36/50
935/935 [==============================] - 75s 80ms/step - loss: 32.4289 - val_loss: 63.7792 - lr: 1.0000e-05
Epoch 37/50
935/935 [===============

2024-03-31 14:07:08.189798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


117/117 [==============================] - 10s 89ms/step - loss: 60.9983
Test Loss: 60.998348236083984


In [11]:
from PIL import Image
import numpy as np

# Load and preprocess 
def preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# image to test
image_path = '/home/jovyan/shared/sampled_data/test_data/00009A02.jpg' 


input_image = preprocess_image(image_path)

predictions = model.predict(input_image)


predicted_age = predictions[0][0]
print("Predicted Age:", predicted_age)


1/1 [==============================] - 0s 33ms/step
Predicted Age: 2.0834982
